In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tifffile as tiff 
from skimage.measure import label, regionprops


In [ ]:
df = pd.read_csv("../input/hubmap-organ-segmentation/train.csv")
df['image_path'] = '../input/hubmap-organ-segmentation/train_images/'
df['image_path'] = df['image_path'].str.cat(df['id'].astype(str))
df['image_path'] = df['image_path'] + '.tiff'
df.head()

In [ ]:
def rleToMask(rleString,height,width):
  rows,cols = height,width
  rleNumbers = [int(numstring) for numstring in rleString.split(' ')]
  rlePairs = np.array(rleNumbers).reshape(-1,2)
  img = np.zeros(rows*cols,dtype=np.uint8)
  for index,length in rlePairs:
    index -= 1
    img[index:index+length] = 255
  img = img.reshape(cols,rows)
  img = img.T
  return img

In [ ]:
id_image = 10044

BASE_PATH = "../input/hubmap-organ-segmentation/"
img_1 = tiff.imread(BASE_PATH + "train_images/" + str(id_image) + ".tiff")
print(img_1.shape)


In [ ]:
mask_1 = rleToMask(df[df["id"]==10044]["rle"].iloc[-1], img_1.shape[1], img_1.shape[0])
mask_1.shape

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(img_1)
plt.imshow(mask_1, cmap='coolwarm', alpha=0.5)

In [ ]:
## Identify all the coordinates in this image
lbl_0 = label(mask_1) 
props = regionprops(lbl_0)
len(props)  

In [ ]:
bboxes = [] ## Convert all the 9 items into bounding boxes so we can save these images for training
for prop in props:
    bboxes.append([prop.bbox[0] - 30, prop.bbox[1] - 30, 
                   prop.bbox[2] + 30, prop.bbox[3] + 30]) ## Adding a little bit of extra image run

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img_1[bboxes[1][0]:bboxes[1][2], bboxes[1][1]:bboxes[1][3], :])
plt.imshow(mask_1[bboxes[1][0]:bboxes[1][2], bboxes[1][1]:bboxes[1][3]], alpha=0.5, cmap='plasma')
plt.show()

In [ ]:
bboxes

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(20, 8))
val = 0
for i in range(2):
    for j in range(4):
        axes[i, j].imshow(img_1[bboxes[val][0]:bboxes[val][2], bboxes[val][1]:bboxes[val][3], :]); val += 1
        axes[i, j].axis('off')
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(20, 8))
val = 0
for i in range(2):
    for j in range(4):
        axes[i, j].imshow(img_1[bboxes[val][0]:bboxes[val][2], bboxes[val][1]:bboxes[val][3], :])
        axes[i, j].imshow(mask_1[bboxes[val][0]:bboxes[val][2], bboxes[val][1]:bboxes[val][3]], alpha=0.5, cmap='plasma')
        axes[i, j].axis('off'); val += 1
plt.show()